## 1. Import flights data for selected airports for 6 days -- 22nd - 27th Dec, 2025 from API and store it in csv file.

In [1]:
!pip3 install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Repeatedly executed for fetching data

In [ ]:
import requests
import pandas as pd
import json

airportsIATA=['DEL','BOM','BLR','MAA','HYD','IXE','VNS','AMD', 'LHR', 'JFK', 'DXB', 'AUH', 'CDG','LAX','SFO']
baseAPIUrl = "https://aerodatabox.p.rapidapi.com/flights/airports/iata/"
querystring = {"offsetMinutes":"-7200","durationMinutes":"720","withLeg":"true","direction":"Both","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"x-rapidapi-key": "0e976473ddmshbf9dfa730c5cedep1248d4jsnb28d82cac028",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

flightsData=pd.DataFrame()

for iata in airportsIATA:
    url = baseAPIUrl + iata
    
    try:
        response = requests.get(url, headers=headers, params=querystring)

        df_arrivals = pd.json_normalize(response.json()["arrivals"])
        df_departures = pd.json_normalize(response.json()["departures"])
        
        #df_arrivals.to_csv(r"C:\Guvi\Data\arrivalData.csv", index=False)
        #df_departures.to_csv(r"C:\Guvi\Data\departureData.csv", index=False)
        
        df_departures['origin'] = iata
        df_departures['destination'] = df_departures['arrival.airport.iata']
        
        df_arrivals['origin'] = df_arrivals['departure.airport.iata']
        df_arrivals['destination'] = iata

        flightsData = pd.concat([flightsData,df_departures, df_arrivals], ignore_index=True)
    except:
        print("Error thrown at iata:" + iata)
        continue

cond = (flightsData['origin'].isin(airportsIATA) & flightsData['destination'].isin(airportsIATA)) 

flightsData = flightsData[cond]
flightsData.to_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\flightsData1.csv", index=False)

flightsData.head()

## 2. Merge the data with previously downloaded data 

In [ ]:
flight1 = pd.read_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\finalFlightsData.csv")
flight2 = pd.read_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\flightsData1.csv")

flight = pd.concat([flight1, flight2]).drop_duplicates(keep=False)
flight.to_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\finalFlightsData2.csv", index=False)

 ## 3. Format data in csv

In [13]:
import pandas as pd
flightsData = pd.read_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\finalFlightsData2.csv")

#Datetime coversion
date_cols = ['departure.scheduledTime.utc','departure.revisedTime.utc',
           'arrival.scheduledTime.utc','arrival.revisedTime.utc']
flightsData[date_cols] = flightsData[date_cols].apply(pd.to_datetime, errors='coerce')

#Select required columns
reqCols = ['number','status','airline.name','airline.iata','aircraft.reg','origin','destination',
           'departure.scheduledTime.utc','departure.scheduledTime.local', 'departure.revisedTime.utc',
           'departure.revisedTime.local','arrival.scheduledTime.utc','arrival.scheduledTime.local',
           'arrival.revisedTime.utc','arrival.revisedTime.local']

flightsData = flightsData[reqCols]
flightsData
#list(flightsData[reqCols].itertuples(index=False,name=None ))

number    status       airline.name airline.iata aircraft.reg origin  \
0      6E 434   Unknown             IndiGo           6E          NaN    DEL   
1      AI 179  Departed          Air India           AI       VT-ALP    DEL   
2     AI 2899   Unknown          Air India           AI       VT-TNP    DEL   
3      AI 143  Departed          Air India           AI       VT-NAC    DEL   
4     IX 1251   Unknown  Air India Express           IX          NaN    DEL   
...       ...       ...                ...          ...          ...    ...   
5233  QR 2141   Arrived              Qatar           QR       N644QX    LAX   
5234  HA 6207   Arrived           Hawaiian           HA       N644QX    LAX   
5235  AS 2037   Arrived             Alaska           AS       N644QX    LAX   
5236  TN 2457   Arrived     Air Tahiti Nui           TN       N644QX    LAX   
5237  BA 7609   Arrived            British           BA       N644QX    LAX   

     destination departure.scheduledTime.utc departure.scheduledTime.local  \
0            MAA   2025-12-23 06:50:00+00:00        2025-12-23 12:20+05:30   
1            SFO   2025-12-23 07:00:00+00:00        2025-12-23 12:30+05:30   
2            HYD   2025-12-23 07:00:00+00:00        2025-12-23 12:30+05:30   
3            CDG   2025-12-23 07:05:00+00:00        2025-12-23 12:35+05:30   
4            VNS   2025-12-23 07:10:00+00:00        2025-12-23 12:40+05:30   
...          ...                         ...                           ...   
5233         SFO   2025-12-27 02:07:00+00:00        2025-12-26 18:07-08:00   
5234         SFO   2025-12-27 02:07:00+00:00        2025-12-26 18:07-08:00   
5235         SFO   2025-12-27 02:07:00+00:00        2025-12-26 18:07-08:00   
5236         SFO   2025-12-27 02:07:00+00:00        2025-12-26 18:07-08:00   
5237         SFO   2025-12-27 02:07:00+00:00        2025-12-26 18:07-08:00   

     departure.revisedTime.utc departure.revisedTime.local  \
0                          NaT                         NaN   
1                          NaT                         NaN   
2                          NaT                         NaN   
3                          NaT                         NaN   
4                          NaT                         NaN   
...                        ...                         ...   
5233 2025-12-27 03:12:00+00:00      2025-12-26 19:12-08:00   
5234 2025-12-27 03:12:00+00:00      2025-12-26 19:12-08:00   
5235 2025-12-27 03:12:00+00:00      2025-12-26 19:12-08:00   
5236 2025-12-27 03:12:00+00:00      2025-12-26 19:12-08:00   
5237 2025-12-27 03:12:00+00:00      2025-12-26 19:12-08:00   

     arrival.scheduledTime.utc arrival.scheduledTime.local  \
0    2025-12-23 09:50:00+00:00      2025-12-23 15:20+05:30   
1    2025-12-23 22:30:00+00:00      2025-12-23 14:30-08:00   
2    2025-12-23 09:20:00+00:00      2025-12-23 14:50+05:30   
3    2025-12-23 17:45:00+00:00      2025-12-23 18:45+01:00   
4    2025-12-23 08:45:00+00:00      2025-12-23 14:15+05:30   
...                        ...                         ...   
5233 2025-12-27 03:36:00+00:00      2025-12-26 19:36-08:00   
5234 2025-12-27 03:36:00+00:00      2025-12-26 19:36-08:00   
5235 2025-12-27 03:36:00+00:00      2025-12-26 19:36-08:00   
5236 2025-12-27 03:36:00+00:00      2025-12-26 19:36-08:00   
5237 2025-12-27 03:36:00+00:00      2025-12-26 19:36-08:00   

       arrival.revisedTime.utc arrival.revisedTime.local  
0                          NaT                       NaN  
1    2025-12-24 02:39:00+00:00    2025-12-23 18:39-08:00  
2                          NaT                       NaN  
3    2025-12-23 17:38:00+00:00    2025-12-23 18:38+01:00  
4                          NaT                       NaN  
...                        ...                       ...  
5233 2025-12-27 04:12:00+00:00    2025-12-26 20:12-08:00  
5234 2025-12-27 04:12:00+00:00    2025-12-26 20:12-08:00  
5235 2025-12-27 04:12:00+00:00    2025-12-26 20:12-08:00  
5236 2025-12-27 04:12:00+00:00    2025-12

#### 4. Store data in MySql database

In [2]:
import mysql.connector
import pandas as pd

connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "0123456789",
    database = "FlightDB"
)
cursor = connection.cursor()

##Format data
flightsData = pd.read_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\finalFlightsData2.csv")

#Datetime coversion
# date_cols = ['departure.scheduledTime.utc','departure.revisedTime.utc',
#            'arrival.scheduledTime.utc','arrival.revisedTime.utc']
# flightsData[date_cols] = flightsData[date_cols].apply(pd.to_datetime, errors='coerce')

#Select required columns
reqCols = ['number','status','airline.name','airline.iata','aircraft.reg','origin','destination',
           'departure.scheduledTime.utc','departure.scheduledTime.local', 'departure.revisedTime.utc',
           'departure.revisedTime.local','arrival.scheduledTime.utc','arrival.scheduledTime.local',
           'arrival.revisedTime.utc','arrival.revisedTime.local']

data = list(flightsData[reqCols].itertuples(index=False,name=None ))
# cursor.execute("drop table Flights")
# connection.commit()
flightCreatequery = """CREATE TABLE IF NOT EXISTS Flights(
            number VARCHAR(10), 
            status VARCHAR(20), 
            airlineName VARCHAR(100), 
            airlineIata VARCHAR(10), 
            aircraftReg varchar(10),
            origin varchar(10),
            destination varchar(10),
            departureSchUTC VARCHAR(100),
            departureSchLocal VARCHAR(100),
            departureRevUTC VARCHAR(100),
            departureRevLocal VARCHAR(100),
            arrivalSchUTC VARCHAR(100),
            arrivalSchLocal VARCHAR(100),
            arrivalRevUTC VARCHAR(100),
            arrivalRevLocal VARCHAR(100),
            foreign key(origin) references Airports(iata),
            foreign key(destination) references Airports(iata))"""
            
cursor.execute(flightCreatequery)

flightInsertQuery = f"""INSERT INTO Flights (number, status, airlineName, airlineIata, aircraftReg, origin, destination,
                        departureSchUTC, departureSchLocal, departureRevUTC, departureRevLocal, arrivalSchUTC,
                        arrivalSchLocal,arrivalRevUTC,arrivalRevLocal ) 
                        VALUES ( %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s )"""
cursor.executemany(flightInsertQuery, data)
connection.commit()

In [6]:
flightInsertQuery = f"""INSERT INTO Flights (number, status, airlineName, airlineIata, aircraftReg, origin, detination,
                        departureSchUTC, departureSchLocal, departureRevUTC, departureRevLocal, arrivalSchUTC,
                        arrivalSchLocal,arrivalRevUTC,arrivalRevLocal ) 
                        VALUES ( {','.join(['%s']*15)} )"""

flightInsertQuery

'INSERT INTO Flights (number, status, airlineName, airlineIata, aircraftReg, origin, detination,\n                        departureSchUTC, departureSchLocal, departureRevUTC, departureRevLocal, arrivalSchUTC,\n                        arrivalSchLocal,arrivalRevUTC,arrivalRevLocal ) \n                        VALUES ( %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s )'

In [1]:
import mysql.connector
import pandas as pd

connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "0123456789",
    database = "FlightDB"
)
cursor = connection.cursor()
cursor.execute("drop table Flights")
connection.commit()